In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os
import random
import struct
import subprocess
import sys
from matplotlib.patches import Rectangle

In [2]:
base_dir = os.getcwd()
exec_dir = '..'

In [3]:
# Returns all records whose location is in the given range.
def range_query(df, lx, ly, ux, uy):
    return (df[(df.x >= lx) & (df.x <= ux) & (df.y >= ly) & (df.y <= uy)])

# Returns the bounding rectangle of a Pandas dataframe.
def bounding_rect(df):
    return (np.min(df['x']), np.min(df['y']), np.max(df['x']), np.max(df['y']))

# Generates a random rectangle inside the given interval.
def random_rect(lx, ly, ux, uy):
    # Generate a random lower-left vertex.
    px = np.random.randint(lx, ux+1)
    py = np.random.randint(ly, uy+1)
    width = np.random.randint(0, ux-px+1)
    height = np.random.randint(0, uy-py+1)
    return (px, py, px+width, py+height)

def to_string(rect):
    return '({}, {}, {}, {})'.format(rect[0],rect[1],rect[2],rect[3])

def queries_with_fraction(df, fmin, fmax):
    return (df[(df.fraction >= fmin) & (df.fraction <= fmax)]).sort_values('fraction')

In [4]:
df = pd.read_csv('data/crash_data.csv')
df.head()

,ID,Year,Month,Day,Time,x,y
0,2012-1-27/05/2021,2012,January,Sunday,04:30 pm,133065971,167179587
1,2012-2-27/05/2021,2012,January,Sunday,09:10 am,132940015,166846266
2,2012-3-27/05/2021,2012,January,Wednesday,11:30 am,131374822,162424128
3,2012-4-27/05/2021,2012,January,Wednesday,10:20 am,132532677,167242555
4,2012-5-27/05/2021,2012,January,Wednesday,03:30 pm,132605645,167302842


# Query generation

We construct a data set of $10^6$ random queries on the data set with $m=30000$ points.

For each query, we measure the number of records that satisfy it.

This is done by means of our <code>QueryGenerator</code> C++ program.

Then we consider fractions from 10% to 90%. For each fraction we select 50 queries with that fraction and create separate data sets.

In [5]:
query_file = 'data/random_queries.csv'
query_df = pd.read_csv(query_file)
for i in range(1, 10):
    size = len(queries_with_fraction(query_df, i/10-0.01, i/10+0.01))
    print('Fraction: {}%\tQueries: {}'.format(i*10, size))

Fraction: 10%	Queries: 1696
Fraction: 20%	Queries: 771
Fraction: 30%	Queries: 461
Fraction: 40%	Queries: 339
Fraction: 50%	Queries: 332
Fraction: 60%	Queries: 397
Fraction: 70%	Queries: 532
Fraction: 80%	Queries: 1009
Fraction: 90%	Queries: 7071


# Fraction of records (10%-90%)

In [16]:
m=30000
c=16
input_file = '{}/data/crash_data_{}.csv'.format(base_dir, m)
output_file = '{}/test_query_verification/test_fraction.csv'.format(base_dir)

In [17]:
f = open(output_file, 'w')
f.write('fraction,avg_returned,avg_matching,avg_query,avg_verif\n')

os.chdir(exec_dir)

for i in range(1, 10):
    print('Testing fraction = {}%...'.format(i*10))
    query_file = '{}/data/random_queries_{}.csv'.format(base_dir, i*10)
    params = ['./TestQuery', input_file, query_file, str(c)]
    p = subprocess.run(params, capture_output=True, check=True)
    lines = p.stdout.decode('utf-8').splitlines()
    avg_retur = int((lines[1].split(':'))[1])
    avg_match = int((lines[2].split(':'))[1])
    avg_query = int((lines[3].split(':'))[1])
    avg_verif = int((lines[4].split(':'))[1])
    f.write('{},{},{},{},{}\n'.format(i*10,avg_retur,avg_match,avg_query,avg_verif))
f.close()

os.chdir(base_dir)
print('Done!')

Testing fraction = 10%...
Testing fraction = 20%...
Testing fraction = 30%...
Testing fraction = 40%...
Testing fraction = 50%...
Testing fraction = 60%...
Testing fraction = 70%...
Testing fraction = 80%...
Testing fraction = 90%...
Done!


In [18]:
df=pd.read_csv('test_query_verification/test_fraction.csv')
# Convert returned and matching columns to integer type.
df = df.astype({'fraction':str, 'avg_returned': int, 'avg_matching': int})
# Add % to fraction column.
df['fraction'] = df['fraction'] + '%'
# Compute noise
df['noise'] = 1-(df['avg_matching']/df['avg_returned'])
#df['noise'] = (df['avg_returned']-df['avg_matching'])/(df['avg_returned']-2*df['avg_matching']+30000)
# Divide by 1000 to convert microseconds to milliseconds.
df['avg_query'] /= 1000
df['avg_verif'] /= 1000
# Round the execution times to 3 decimal places.
df = df.round({'noise':3, 'avg_query': 3, 'avg_verif': 3})
# Reorder columns.
df = df[['fraction','avg_returned','avg_matching','noise','avg_query','avg_verif']]
print(df.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
fraction &  avg\_returned &  avg\_matching &  noise &  avg\_query &  avg\_verif \\
\midrule
     10\% &          6186 &          2978 &  0.519 &      0.120 &      2.810 \\
     20\% &          9419 &          6014 &  0.362 &      0.187 &      4.429 \\
     30\% &         12952 &          8991 &  0.306 &      0.247 &      6.023 \\
     40\% &         16512 &         12013 &  0.272 &      0.314 &      7.629 \\
     50\% &         19132 &         15033 &  0.214 &      0.360 &      8.871 \\
     60\% &         21377 &         18019 &  0.157 &      0.398 &     10.099 \\
     70\% &         24440 &         20970 &  0.142 &      0.449 &     11.484 \\
     80\% &         26393 &         24014 &  0.090 &      0.515 &     12.438 \\
     90\% &         29001 &         27013 &  0.069 &      0.528 &     13.551 \\
\bottomrule
\end{tabular}



# Low fractions (1%-9%)

Let's see how many queries we can select for each fraction from 1% to 9%.

In [19]:
query_df = pd.read_csv('data/random_queries.csv')
for i in range(1, 10):
    size = len(queries_with_fraction(query_df, i/100-0.001, i/100+0.001))
    print('Fraction: {}%\tQueries: {}'.format(i, size))

Fraction: 1%	Queries: 18209
Fraction: 2%	Queries: 6504
Fraction: 3%	Queries: 4254
Fraction: 4%	Queries: 2659
Fraction: 5%	Queries: 1305
Fraction: 6%	Queries: 567
Fraction: 7%	Queries: 372
Fraction: 8%	Queries: 249
Fraction: 9%	Queries: 198


From these results, we see that we can select once again 100 queries for each fraction.

Then we can run our simulation.

In [20]:
m=30000
c=16
input_file = '{}/data/crash_data_{}.csv'.format(base_dir, m)
output_file = '{}/test_query_verification/test_fraction_low.csv'.format(base_dir)

In [21]:
f = open(output_file, 'w')
f.write('fraction,avg_returned,avg_matching,avg_query,avg_verif\n')

os.chdir(exec_dir)

for i in range(1, 10):
    print('Testing fraction = {}%...'.format(i))
    query_file = '{}/data/random_queries_{}.csv'.format(base_dir, i)
    params = ['./TestQuery', input_file, query_file, str(c)]
    p = subprocess.run(params, capture_output=True, check=True)
    lines = p.stdout.decode('utf-8').splitlines()
    avg_retur = int((lines[1].split(':'))[1])
    avg_match = int((lines[2].split(':'))[1])
    avg_query = int((lines[3].split(':'))[1])
    avg_verif = int((lines[4].split(':'))[1])
    f.write('{},{},{},{},{}\n'.format(i,avg_retur,avg_match,avg_query,avg_verif))
    
f.close()

os.chdir(base_dir)
print('Done!')

Testing fraction = 1%...
Testing fraction = 2%...
Testing fraction = 3%...
Testing fraction = 4%...
Testing fraction = 5%...
Testing fraction = 6%...
Testing fraction = 7%...
Testing fraction = 8%...
Testing fraction = 9%...
Done!


In [22]:
df=pd.read_csv('test_query_verification/test_fraction_low.csv')
# Convert returned and matching columns to integer type.
df = df.astype({'fraction': str, 'avg_returned': int, 'avg_matching': int})
# Add % to fraction column.
df['fraction'] = df['fraction'] + '%'
# Compute noise.
#df['noise'] = (df['avg_returned']-df['avg_matching'])/(df['avg_returned']-2*df['avg_matching']+30000)
df['noise'] = 1-(df['avg_matching']/df['avg_returned'])
# Divide by 1000 to convert microseconds to milliseconds.
df['avg_query'] /= 1000
df['avg_verif'] /= 1000
# Round the execution times to 3 decimal places.
df = df.round({'noise':3, 'avg_query': 3, 'avg_verif': 3})
# Reorder columns.
df = df[['fraction','avg_returned','avg_matching','noise','avg_query','avg_verif']]
print(df.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
fraction &  avg\_returned &  avg\_matching &  noise &  avg\_query &  avg\_verif \\
\midrule
      1\% &          1449 &           300 &  0.793 &      0.033 &      0.631 \\
      2\% &          2003 &           601 &  0.700 &      0.044 &      0.885 \\
      3\% &          2315 &           897 &  0.613 &      0.047 &      1.005 \\
      4\% &          2603 &          1198 &  0.540 &      0.051 &      1.112 \\
      5\% &          3105 &          1499 &  0.517 &      0.062 &      1.363 \\
      6\% &          3701 &          1798 &  0.514 &      0.084 &      1.897 \\
      7\% &          4129 &          2099 &  0.492 &      0.079 &      1.832 \\
      8\% &          4578 &          2399 &  0.476 &      0.099 &      2.257 \\
      9\% &          5528 &          2701 &  0.511 &      0.111 &      2.459 \\
\bottomrule
\end{tabular}



# Capacity

Pick 100 queries with $f_Q \approx 20\%$. 
For each capacity value $c$ build a MR-tree on data set $\mathcal{D}$.
Then run Q and V on the 100 queries and compute an average.

In [5]:
n_records = 30000
query_file = 'test/data/random_queries_test_capacity.csv'
input_file = 'test/data/crash_data_{}.csv'.format(n_records)
output_file = 'test_query_verification/test_capacity.csv'
capacities = [4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]

f = open(output_file, 'w')
f.write('capacity,avg_returned,avg_matching,avg_query,avg_verif\n')

os.chdir(exec_dir)

for c in capacities:
    # Run the C++ implementation.
    print('Testing capacity = {}...'.format(c))
    params = ['./TestQuery', input_file, query_file, str(c)]
    p = subprocess.run(params, capture_output=True, check=True)
    lines = p.stdout.decode('utf-8').splitlines()
    avg_retur = int((lines[1].split(':'))[1])
    avg_match = int((lines[2].split(':'))[1])
    avg_query = int((lines[3].split(':'))[1])
    avg_verif = int((lines[4].split(':'))[1])
    f.write('{},{},{},{},{}\n'.format(c,avg_retur,avg_match,avg_query,avg_verif))

f.close()

os.chdir(base_dir)
print('Done!')

Testing capacity = 4...


CalledProcessError: Command '['./TestQuery', 'test/data/crash_data_30000.csv', 'test/data/random_queries_test_capacity.csv', '4']' died with <Signals.SIGSEGV: 11>.

In [5]:
df=pd.read_csv('test_query_verification/test_capacity.csv')
# Convert returned and matching columns to integer type.
df = df.astype({'avg_returned': int, 'avg_matching': int})
# Divide by 1000 to convert microseconds to milliseconds.
df['avg_query'] /= 1000
df['avg_verif'] /= 1000
# Compute noise.
df['noise'] = 1-(df['avg_matching']/df['avg_returned'])
# Round the execution times to 3 decimal places.
df = df.round({'noise': 3, 'avg_query': 3, 'avg_verif': 3})
df = df[['capacity','avg_returned','avg_matching','noise','avg_query','avg_verif']]
print(df.to_latex(index=False))

\begin{tabular}{rrrrrr}
\toprule
 capacity &  avg\_returned &  avg\_matching &  noise &  avg\_query &  avg\_verif \\
\midrule
        4 &          6604 &          5985 &  0.094 &      1.236 &      9.506 \\
        8 &          7544 &          5985 &  0.207 &      0.935 &      6.443 \\
       16 &          9436 &          5985 &  0.366 &      0.851 &      6.091 \\
       32 &         13408 &          5985 &  0.554 &      1.069 &      6.678 \\
       64 &         18722 &          5985 &  0.680 &      1.328 &      8.425 \\
      128 &         21889 &          5985 &  0.727 &      1.489 &      9.222 \\
      256 &         30000 &          5985 &  0.800 &      2.004 &     11.099 \\
      512 &         30000 &          5985 &  0.800 &      2.110 &     11.126 \\
     1024 &         30000 &          5985 &  0.800 &      2.017 &     11.025 \\
     2048 &         30000 &          5985 &  0.800 &      2.006 &     10.918 \\
\bottomrule
\end{tabular}

